# HVNM Limiting Cases and Comparisons

**Hybrid Vitrimer-Nanocomposite Model — Factory methods and model reduction**

## Learning Objectives

- Use factory methods to create standard limiting cases
- Verify φ=0 recovers HVM (no nanoparticle effects)
- Compare SAOS across different model variants
- Visualize Guth-Gold amplification X(φ) scaling
- Analyze network fraction decomposition

**Runtime:** ~1 minute

In [ ]:
# Google Colab setup (run only if on Colab)
try:
    import google.colab
    IN_COLAB = True
    !pip install -q rheojax
except ImportError:
    IN_COLAB = False
    print("Running locally")

In [ ]:
# Imports
from rheojax.core.jax_config import safe_import_jax, verify_float64
jax, jnp = safe_import_jax()
verify_float64()

from rheojax.models import HVNMLocal, HVMLocal
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

plt.rcParams.update({
    'figure.dpi': 100,
    'font.size': 10,
    'axes.labelsize': 11,
    'axes.titlesize': 12,
    'legend.fontsize': 9
})

print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

## Theory: Limiting Cases

The HVNM model provides factory methods for standard configurations:

1. **unfilled_vitrimer**: φ=0, all β_i=0 → Standard HVM (G_P + G_E + G_D)
2. **filled_elastomer**: G_E=0, G_D=0 → Filled permanent network (G_P·X_I + G_I·X_I)
3. **partial_vitrimer_nc**: G_P > 0, G_E > 0, G_D > 0, φ > 0 → Full HVNM
4. **conventional_filled_rubber**: High β_I, low G_E → Elastomer-like behavior
5. **matrix_only_exchange**: β_I=0, β_E=0, β_D=0 → No immobilization, only matrix networks

**Key relationship**: At φ=0, X_I = X_E = X_D = 1, so HVNM → HVM.

**Guth-Gold amplification**:
$$X(\phi) = 1 + 2.5\phi + 14.1\phi^2$$

This provides quadratic stiffening with φ.

In [ ]:
# Create 5 limiting cases using factory methods

# 1. Unfilled vitrimer (φ=0)
model_unfilled = HVNMLocal.unfilled_vitrimer(
    G_P=5000.0, G_E=3000.0, G_D=1000.0,
    nu_0=1e10, E_a=80e3, T=350.0, k_d_D=10.0
)

# 2. Filled elastomer (no vitrimer networks)
model_elastomer = HVNMLocal.filled_elastomer(
    G_P=5000.0, phi=0.1
)

# 3. Partial vitrimer nanocomposite (full HVNM)
model_partial = HVNMLocal.partial_vitrimer_nc(
    G_P=5000.0, G_E=3000.0, phi=0.1,
    nu_0=1e10, E_a=80e3, T=350.0, beta_I=3.0
)

# 4. Conventional filled rubber
model_rubber = HVNMLocal.conventional_filled_rubber(
    G_P=5000.0, G_D=1000.0, phi=0.15, k_d_D=10.0
)

# 5. Matrix-only exchange (no immobilization)
model_matrix = HVNMLocal.matrix_only_exchange(
    G_P=5000.0, G_E=3000.0, phi=0.1,
    nu_0=1e10, E_a=80e3, T=350.0, k_d_D=10.0
)

models = {
    "Unfilled Vitrimer (φ=0)": model_unfilled,
    "Filled Elastomer": model_elastomer,
    "Partial Vitrimer NC": model_partial,
    "Conventional Filled Rubber": model_rubber,
    "Matrix-Only Exchange": model_matrix
}

print("Factory Methods Created:")
for name, m in models.items():
    phi_val = m.phi
    X = 1.0 + 2.5*phi_val + 14.1*phi_val**2
    print(f"  {name:30s}: φ={phi_val:.2f}, X(φ)={X:.3f}")

In [ ]:
# SAOS comparison across limiting cases
omega = np.logspace(-2, 3, 100)
colors = ['blue', 'green', 'red', 'orange', 'purple']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

print("\nComputing SAOS for each limiting case...")
for (name, model), color in zip(models.items(), colors):
    G_prime, G_double_prime = model.predict_saos(omega)
    ax1.loglog(omega, G_prime, "-", lw=1.5, color=color, label=name)
    ax2.loglog(omega, G_double_prime, "--", lw=1.5, color=color, label=name)
    print(f"  {name:30s}: G'(ω=1) = {G_prime[50]:.1f} Pa, G''(ω=1) = {G_double_prime[50]:.1f} Pa")

ax1.set_xlabel("ω [rad/s]")
ax1.set_ylabel("G' [Pa]")
ax1.set_title("Storage Modulus")
ax1.legend(fontsize=8)
ax1.grid(True, alpha=0.3, which="both")

ax2.set_xlabel("ω [rad/s]")
ax2.set_ylabel("G'' [Pa]")
ax2.set_title("Loss Modulus")
ax2.legend(fontsize=8)
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Verify φ=0 recovers HVM
print("\nVerifying HVNM(φ=0) == HVM...")

# HVM reference
hvm = HVMLocal()
hvm.parameters.set_value("G_P", 5000.0)
hvm.parameters.set_value("G_E", 3000.0)
hvm.parameters.set_value("G_D", 1000.0)
hvm.parameters.set_value("T", 350.0)
hvm.parameters.set_value("k_d_D", 10.0)
hvm.parameters.set_value("nu_0", 1e10)
hvm.parameters.set_value("E_a", 80e3)
hvm.parameters.set_value("V_act", 1e-5)

# HVNM with φ=0 (use unfilled_vitrimer factory)
hvnm_zero = HVNMLocal.unfilled_vitrimer(
    G_P=5000.0, G_E=3000.0, G_D=1000.0,
    nu_0=1e10, E_a=80e3, V_act=1e-5, T=350.0, k_d_D=10.0
)

# Compare SAOS
G_p_hvm, G_pp_hvm = hvm.predict_saos(omega)
G_p_hvnm, G_pp_hvnm = hvnm_zero.predict_saos(omega)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.loglog(omega, G_p_hvm, "b-", lw=2, label="HVM")
ax1.loglog(omega, G_p_hvnm, "r--", lw=1.5, label="HVNM (φ=0)")
ax1.set_xlabel("ω [rad/s]")
ax1.set_ylabel("G' [Pa]")
ax1.set_title("Storage Modulus: HVM vs HVNM(φ=0)")
ax1.legend()
ax1.grid(True, alpha=0.3, which="both")

ax2.loglog(omega, G_pp_hvm, "b-", lw=2, label="HVM")
ax2.loglog(omega, G_pp_hvnm, "r--", lw=1.5, label="HVNM (φ=0)")
ax2.set_xlabel("ω [rad/s]")
ax2.set_ylabel("G'' [Pa]")
ax2.set_title("Loss Modulus: HVM vs HVNM(φ=0)")
ax2.legend()
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

# Compute relative error
rel_err_Gp = np.abs(G_p_hvm - G_p_hvnm) / (np.abs(G_p_hvm) + 1e-10)
rel_err_Gpp = np.abs(G_pp_hvm - G_pp_hvnm) / (np.abs(G_pp_hvm) + 1e-10)

print(f"\nRelative error in G': max = {rel_err_Gp.max():.2e}, mean = {rel_err_Gp.mean():.2e}")
print(f"Relative error in G'': max = {rel_err_Gpp.max():.2e}, mean = {rel_err_Gpp.mean():.2e}")
print("\n→ HVNM(φ=0) recovers HVM (curves overlap within numerical precision)")

In [ ]:
# Guth-Gold amplification sweep
phi_sweep = np.linspace(0, 0.2, 50)
X_sweep = 1.0 + 2.5*phi_sweep + 14.1*phi_sweep**2
G_P = 5000.0

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# X(φ) vs φ
ax1.plot(phi_sweep, X_sweep, "b-", lw=2)
ax1.scatter([0, 0.05, 0.1, 0.15, 0.2], 
            [1.0 + 2.5*p + 14.1*p**2 for p in [0, 0.05, 0.1, 0.15, 0.2]],
            color="red", s=80, zorder=5, label="Sample points")
ax1.set_xlabel("φ (NP volume fraction)")
ax1.set_ylabel("X(φ) (Amplification factor)")
ax1.set_title("Guth-Gold Amplification")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Effective modulus G_P*X(φ) vs φ
ax2.plot(phi_sweep, G_P * X_sweep, "g-", lw=2)
ax2.set_xlabel("φ (NP volume fraction)")
ax2.set_ylabel("G_P·X(φ) [Pa]")
ax2.set_title("Effective P-Network Modulus")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

print("\nGuth-Gold Amplification X(φ):")
print(f"{'φ':>6} {'X(φ)':>8} {'G_P·X':>12}")
print("-" * 30)
for p in [0.0, 0.05, 0.1, 0.15, 0.2]:
    X = 1.0 + 2.5*p + 14.1*p**2
    print(f"{p:>6.2f} {X:>8.3f} {G_P*X:>12.1f}")
print("\n→ Quadratic stiffening: X(φ) = 1 + 2.5φ + 14.1φ²")

In [ ]:
# Network fractions visualization (pie chart)
# For partial vitrimer NC at φ=0.1, β_I=3.0
phi_val = 0.1
beta_I = 3.0
X_I = 1.0 + 2.5*phi_val + 14.1*phi_val**2

# Effective moduli
G_P_eff = 5000.0 * X_I
G_E_eff = 3000.0 * 1.0  # X_E = 1 (assumed, or compute cascade)
G_I_eff = beta_I * phi_val * 5000.0 * X_I  # G_I = β_I·φ·G_P·X_I
G_D_eff = 1000.0 * 1.0  # X_D = 1 (assumed)

labels = ['P-network\n(permanent)', 'E-network\n(exchangeable)', 
          'I-network\n(immobilized)', 'D-network\n(dangling)']
sizes = [G_P_eff, G_E_eff, G_I_eff, G_D_eff]
colors_pie = ['blue', 'green', 'red', 'orange']
explode = (0.05, 0, 0.05, 0)  # Highlight P and I

fig, ax = plt.subplots(figsize=(8, 8))
ax.pie(sizes, explode=explode, labels=labels, colors=colors_pie,
       autopct='%1.1f%%', shadow=True, startangle=90)
ax.set_title(f"Network Fraction Decomposition\n(φ={phi_val:.1f}, β_I={beta_I:.1f}, X_I={X_I:.3f})",
             fontsize=12)

plt.tight_layout()
display(fig)
plt.close(fig)

print("\nNetwork Contributions (effective moduli):")
print(f"  G_P_eff = G_P·X_I = {G_P_eff:.1f} Pa")
print(f"  G_E_eff = G_E·X_E ≈ {G_E_eff:.1f} Pa")
print(f"  G_I_eff = β_I·φ·G_P·X_I = {G_I_eff:.1f} Pa")
print(f"  G_D_eff = G_D·X_D ≈ {G_D_eff:.1f} Pa")
print(f"\nTotal = {sum(sizes):.1f} Pa")

## Key Takeaways

1. **Factory methods**: Provide standard limiting cases (unfilled, filled elastomer, etc.)
2. **φ=0 recovers HVM**: HVNM reduces to HVM when no nanoparticles are present
3. **Guth-Gold scaling**: X(φ) = 1 + 2.5φ + 14.1φ² provides quadratic stiffening
4. **Network decomposition**: P, E, I, D networks contribute with different amplification factors
5. **Model versatility**: Single HVNM framework captures multiple material classes

## Physical Interpretation

The HVNM model is a **unified framework** for filled vitrimers:

- **Unfilled vitrimer** (φ=0): Standard HVM with P, E, D networks
- **Filled elastomer** (G_E=G_D=0): Permanent network with NP amplification
- **Partial vitrimer NC**: Full HVNM with all four subnetworks
- **Conventional filled rubber**: High immobilization, low exchange
- **Matrix-only exchange**: NPs present but no immobilized chains

The **Guth-Gold amplification** X(φ) = 1 + 2.5φ + 14.1φ² is a classical result from filled elastomer theory (Einstein linear term + Smallwood quadratic correction). It captures:

- **Hydrodynamic effect**: Rigid particles exclude volume → 2.5φ
- **Particle-particle interaction**: Crowding at higher φ → 14.1φ²

At φ=0.1, X_I ≈ 1.39 (39% stiffening). At φ=0.2, X_I ≈ 2.06 (106% stiffening).

The **network fractions** show that for typical parameters (φ=0.1, β_I=3), the I-network contributes ~10-20% of total modulus at low frequencies, while P-network dominates. This matches experimental observations of filled vitrimers.

## Experimental Validation

Use these limiting cases to:

1. **Validate φ-dependence**: Measure SAOS at φ=0, 0.05, 0.1, 0.15, 0.2 → verify Guth-Gold
2. **Identify model class**: Does material behave like unfilled vitrimer, filled elastomer, or partial NC?
3. **Extract parameters**: Fit SAOS, startup, creep to determine G_P, G_E, G_D, β_I
4. **Test TST**: Temperature sweeps (T) and stress sweeps (V_act) validate bond exchange kinetics

## Next Steps

- Fit experimental data to determine which limiting case applies
- Explore temperature-dependent SAOS to extract E_a (Arrhenius)
- Investigate startup transients across limiting cases
- Compare with other nanocomposite models (e.g., percolation-based)
- Extend to nonlocal HVNM for shear banding predictions